In [ ]:
import numpy as np
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath("..")) 

import importlib
import bmc
import data
import inference_utils

importlib.reload(bmc)
importlib.reload(data)
importlib.reload(inference_utils)

from bmc import BayesianModelCombination
from inference_utils import USVt_hat_extraction, gibbs_sampler_simplex
from data import Dataset


models = [
    'AME2020', 'ME2', 'MEdelta', 'PC1', 'NL3S', 'SKMS', 'SKP', 'SLY4',
    'SV', 'UNEDF0', 'UNEDF1', 'UNEDF2', 'FRDM12', 'HFB24', 'BCPM', 'D1M'
]
properties = ["BE", "ChRad"]
domain_keys = ["N", "Z"]

colors = [
    "#1f77b4", # Vivid blue
    "#ff7f0e", # Bright orange
    "#2ca02c", # Rich green
    "#d62728", # Strong red
    "#9467bd", # Deep purple
    # "#8c564b", # Brownish-pink
    "#e377c2", # Pink
    "#7f7f7f", # Medium gray
    "#bcbd22", # Lime green
    "#17becf", # Cyan
    "#393b79", # Dark blue
    "#637939", # Olive green
    "#8c6d31", # Bronze
    # "#843c39", # Dark red
    # "#ad494a", # Reddish brown
    "#d6616b", # Soft red
    "#e7ba52", # Golden yellow
    "#7b4173", # Dark purple
    "#a55194", # Mauve
    "#ce6dbd", # Light purple
]

In [2]:
# Load property DataFrames
dataset = Dataset(r"C:\Users\congn\OneDrive\Desktop\An Le Materials\ModelOrthogonalization\data\selected_data.h5")

In [3]:
property_data = dataset.load_data(models=models, keys=properties, domain_keys=domain_keys) 
print('property data:', property_data.keys())
for prop, df in property_data.items():
    print(f"{prop} DataFrame shape: {df.shape}")
    print(df.head())

[Skipped] Model 'AME2020' missing columns ['ChRad'] for property 'ChRad'.
[Skipped] Model 'UNEDF2' missing columns ['ChRad'] for property 'ChRad'.
[Skipped] Model 'FRDM12' missing columns ['ChRad'] for property 'ChRad'.
[Skipped] Model 'HFB24' missing columns ['ChRad'] for property 'ChRad'.
[Skipped] Model 'BCPM' missing columns ['ChRad'] for property 'ChRad'.
[Skipped] Model 'D1M' missing columns ['ChRad'] for property 'ChRad'.
property data: dict_keys(['BE', 'ChRad'])
BE DataFrame shape: (629, 18)
    N  Z     AME2020      ME2  MEdelta      PC1     NL3S        SKMS  \
0   8  8  127.619315  126.738  129.026  127.455  128.114  128.856436   
1  10  8  139.807766  140.156  141.992  141.423  141.715  144.746257   
2  12  8  151.371414  151.224  152.793  153.215  153.432  158.460613   
3  14  8  162.027188  160.513  161.884  163.303  163.311  170.486446   
4  16  8  168.952452  167.472  168.465  170.768  170.970  178.908577   

          SKP        SLY4          SV      UNEDF0      UNEDF1 

In [4]:
property_data['ChRad']

N    Z    ME2  MEdelta    PC1   NL3S      SKMS       SKP      SLY4  \
0      2    2  2.145    2.081  2.183  2.145  2.115416  2.124712  2.139876   
1      4    2  2.128    2.071  2.136  2.109  2.089149  2.112222  2.106217   
2      6    2  2.142    2.085  2.135  2.116  2.100844  2.131091  2.114904   
3      4    4  2.552    2.512  2.498  2.639  2.427067  2.455516  2.453457   
4      6    4  2.429    2.435  2.431  2.385  2.413912  2.448446  2.438173   
..   ...  ...    ...      ...    ...    ...       ...       ...       ...   
637  154  106  6.097    6.083  6.093  6.099  6.083176  6.094002  6.085548   
638  156  106  6.109    6.097  6.106  6.111  6.092065  6.102800  6.094902   
639  156  108  6.123    6.117  6.122  6.126  6.109944  6.126930  6.115375   
640  158  108  6.136    6.128  6.134  6.138  6.119354  6.135442  6.124092   
641  160  110  6.162    6.158  6.155  6.166  6.149867  6.163781  6.151684   

           SV    UNEDF0    UNEDF1  
0    2.284907  2.082280  2.236110  
1    2.175808  2.060797  2.138773  
2    2.165654  2.088327  2.141098  
3    2.517717  2.403660  2.477630  
4    2.461885  2.387852  2.420674  
..        ...       ...       ...  
637  6.070406  6.073800  6.089964  
638  6.079813  6.083373  6.099768  
639  6.100274  6.103005  6.118907  
640  6.109421  6.112236  6.128383  
641  6.137313  6.141334  6.154904  

[642 rows x 12 columns]

In [5]:
# Use .get_subset() to filter by Z range for BE
filtered_df = dataset.get_subset(
    property_name="BE",
    filters={"Z": (26, 28)},
    models_to_include=['ME2', 'NL3S', 'SKP']  # Optional
)

print("\nFiltered BE data from get_subset:")
print(filtered_df.head())

print("\n========== Testing `view_data` Method ==========")
# 1. View available models and keys
info = dataset.view_data()
print("Available models and keys:")
print(info)

# 2. View full data for a specific model 
print("\nFull DataFrame for model 'ME2':")
df_me2 = dataset.view_data(model_name='ME2')
print(df_me2)

# 3. View 'BE' key across all models
print("\n'BE' values across all models:")
be_values = dataset.view_data(property_name='BE')
print(be_values.head())

# 4. View 'BE' values for model 'SKP'
print("\n'BE' values for model 'SKP':")
be_skp = dataset.view_data(model_name='SKP', property_name='BE')
print(be_skp.head())

# Split data using the updated split_data method
train_data_be, val_data_be, test_data_be = dataset.split_data( 
    data_dict=property_data,
    property_name="BE",
    splitting_algorithm="random",
    train_size=0.7, val_size=0.15, test_size=0.15
)

print("\nTrain data:")
print(train_data_be.head())
print("\nValidation data:")
print(val_data_be.head())
print("\nTest data:")
print(test_data_be.head())

# For BMC, use all model columns except AME2020 (which is used as truth)
models_list = train_data_be.columns.tolist()
#print("\nModel columns for BMC:", models_list)

# Initialize BMC, orthogonalize, train, and predict
bmc = BayesianModelCombination(models_list=models_list, data_dict=property_data, truth_column_name="AME2020") 
bmc.orthogonalize(property="BE", train_df=train_data_be, components_kept=3) 
# bmc.train(training_options={"iterations": 100000, "sampler": 'samplex'}) 
print(f"\nNumber of models used: {bmc.Vt_hat.shape[1]}")


# Predict
# rndm_m, lower_df, median_df, upper_df = bmc.predict2(property="ChRad") 


# print("\nBayesianModelCombination results:")
# print("Predicted mean:", rndm_m)
# print("Predicted upper CI:", upper_df.head())
# print("Predicted median:", median_df.head())
# print("Predicted lower CI:", lower_df.head())

# # Evaluate
# eval_results=bmc.evaluate() #type: ignore 
# print("\nEvaluation results:")
# print(eval_results)


Filtered BE data from get_subset:
     N   Z      ME2     NL3S         SKP
82  22  26  382.766  383.331  387.924188
83  24  26  414.785  414.660  418.538617
84  26  26  443.645  443.142  445.850760
85  28  26  468.403  468.793  470.286541
86  30  26  488.224  487.836  492.065103

========== Testing `view_data` Method ==========
Available models and keys:
{'available_properties': ['BE', 'ChRad'], 'available_models': ['AME2020', 'BCPM', 'D1M', 'FRDM12', 'HFB24', 'ME2', 'MEdelta', 'NL3S', 'PC1', 'SKMS', 'SKP', 'SLY4', 'SV', 'UNEDF0', 'UNEDF1', 'UNEDF2']}

Full DataFrame for model 'ME2':
{'BE':        N    Z       ME2
0      8    8   126.738
1     10    8   140.156
2     12    8   151.224
3     14    8   160.513
4     16    8   167.472
..   ...  ...       ...
624  154  106  1910.765
625  156  106  1925.019
626  156  108  1928.415
627  158  108  1943.010
628  160  110  1960.623

[629 rows x 3 columns], 'ChRad':        N    Z    ME2
0      2    2  2.145
1      4    2  2.128
2      6    2  2

In [11]:
bmc.train(training_options={"iterations": 100000, "sampler": 'simplex', "stepsize": 0.1, "nu0_chosen": 10, "sigma20_chosen" : 0.2}) 

[INFO] Using default value for 'burn': 10000
[INFO] Using default value for 'b_mean_prior': [0. 0. 0.]
[INFO] Using default value for 'b_mean_cov': [[9.98395933e+08 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.93779431e+04 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.43196707e+04]]
we want percentage accepted to be around 20 to 40 percent
percentage actually accepted: 0%


### Workflow: 

1. Initialize Dataset class
2. Use load_data method to load the data
3. Use split_data to get training data
4. Get a list of the models being used (this is needed for BMC initialization)
5. Initialize BMC class
6. Orthogonalize
7. Train
8. Predict
9. Evaluate

In [ ]:
from sampling_utils import rndm_m_random_calculator

preds = test_data_be[models_list].to_numpy()  

samples = bmc.samples  

VT_hat = bmc.Vt_hat  

%time rndm_m_random_calculator(preds, samples, VT_hat)


ValueError: a must be a sequence or an integer, not <class 'NoneType'>

In [ ]:
from sampling_utils import coverage
rndm_m, (lower, median, upper) = rndm_m_random_calculator(preds, samples, VT_hat)
df=bmc.data_dict["BE"]
truth_column_name = bmc.truth_column_name

%time coverage(np.arange(0, 101, 5), rndm_m, df, truth_column=truth_column_name)

ValueError: a must be a sequence or an integer, not <class 'NoneType'>